# Preprocessing and Information Retrieval

Student Name: Shireen Hassan



## Overview

In this task, we will be using documents from a Wall Street Journal text corpus to create a space efficient inverted index capable of fast TF-IDF query processing.


# 1. Preprocessing (2 marks)

For this homework we will be using documents from a Wall Street Journal text corpus. 
The corpus will be downloaded with the commands below. 
Each line contains <i>one</i> document which will be tokenize and stem using tools provided by NLTK.

To download documents from a Wall Street Journal text corpus.

In [1]:
import requests
from pathlib import Path

fname = 'wsta_col_20k.gz'
my_file = Path(fname)
if not my_file.is_file():
    url = 'https://trevorcohn.github.io/comp90042/resources/' + fname
    r = requests.get(url)

    # Save to the current directory
    with open(fname, 'wb') as f:
        f.write(r.content)


read raw documents, one document per line. 

In [2]:
import gzip

raw_docs = []
with gzip.open(fname, 'rt') as f:
    for raw_doc in f:
        raw_docs.append(raw_doc)

print(len(raw_docs))
print(raw_docs[0])

20000
John Blair & Co. is close to an agreement to sell its TV station advertising representation operation and program production unit to an investor group led by James H. Rosenfield, a former CBS Inc. executive, industry sources said. Industry sources put the value of the proposed acquisition at more than $100 million. John Blair was acquired last year by Reliance Capital Group Inc., which has been divesting itself of John Blair's major assets. John Blair represents about 130 local television stations in the placement of national and other advertising. Mr. Rosenfield stepped down as a senior executive vice president of CBS Broadcasting in December 1985 under a CBS early retirement program. Neither Mr. Rosenfield nor officials of John Blair could be reached for comment. 



<b>Preprocessing</b> (1 mark):
*tokenize* each document, *stem* and *lowercase* each token using NLTK `word_tokenize` and `PorterStemmer`, and create a *vocabulary* for all the terms (normalized types). 
Each term will be assigned a unique ID. 
We are not doing any stop word removal. 
Vocabulary will be built as a Python *map*, mapping from all the terms $M$ to their term IDs (integers of $[0..M-1]$).  


In [3]:
import nltk

# processed_docs stores the list of processed docs
processed_docs = []
# vocab contains (term, term id) pairs
vocab = {}
# total_tokens stores the total number of tokens
total_tokens = 0

# TODO: iterate over docs, tokenize, stem and add to vocab and assign ID if new token
stemmer = nltk.stem.PorterStemmer()

for raw_doc in raw_docs:
    
    # norm_doc stores the normalized tokens of a doc
    norm_doc = []
    
    for token in nltk.word_tokenize(raw_doc):
        norm_doc.append(stemmer.stem(token).lower())
        total_tokens += 1   
    for doc_token in norm_doc:
        token_id = len(vocab)
        if doc_token not in vocab.keys():
            vocab[doc_token] = token_id
    
    processed_docs.append(norm_doc)   
    

    
print("Number of documents = {}".format(len(processed_docs)))
print("Number of unique terms = {}".format(len(vocab)))
print("Number of tokens = {}".format(total_tokens))

Number of documents = 20000
Number of unique terms = 103193
Number of tokens = 9140697


<b>For testing:</b>

In [4]:
assert(len(processed_docs) == 20000)

In [5]:
assert(len(vocab) > 100000)

<b>Build a Python `Counter`</b>: (1 mark)
To count the term frequencies of each document. 
For each document, the counter will map the terms to term frequencies. 
All the counters (for all documents) will be stored in a list called *doc_term_freqs*.


In [6]:
from collections import Counter

# doc_term_freqs stores the counters (mapping terms to term frequencies) of all documents
doc_term_freqs = []

# TODO iterate over document and for each document produce the term frequency map and store in the list

###
# Your answer BEGINS HERE
###
for doc in processed_docs:    
    doc_term_freqs.append(Counter(doc))
###
# Your answer ENDS HERE
###

print(len(doc_term_freqs))
print(doc_term_freqs[0])
print(doc_term_freqs[100])

20000
Counter({'.': 6, 'john': 5, 'blair': 5, 'of': 5, 'to': 3, 'rosenfield': 3, ',': 3, 'a': 3, 'cb': 3, 'the': 3, 'an': 2, 'station': 2, 'advertis': 2, 'and': 2, 'program': 2, 'group': 2, 'by': 2, 'inc.': 2, 'execut': 2, 'industri': 2, 'sourc': 2, 'in': 2, 'mr.': 2, '&': 1, 'co.': 1, 'is': 1, 'close': 1, 'agreement': 1, 'sell': 1, 'it': 1, 'tv': 1, 'represent': 1, 'oper': 1, 'product': 1, 'unit': 1, 'investor': 1, 'led': 1, 'jame': 1, 'h.': 1, 'former': 1, 'said': 1, 'put': 1, 'valu': 1, 'propos': 1, 'acquisit': 1, 'at': 1, 'more': 1, 'than': 1, '$': 1, '100': 1, 'million': 1, 'wa': 1, 'acquir': 1, 'last': 1, 'year': 1, 'relianc': 1, 'capit': 1, 'which': 1, 'ha': 1, 'been': 1, 'divest': 1, 'itself': 1, "'s": 1, 'major': 1, 'asset': 1, 'repres': 1, 'about': 1, '130': 1, 'local': 1, 'televis': 1, 'placement': 1, 'nation': 1, 'other': 1, 'step': 1, 'down': 1, 'as': 1, 'senior': 1, 'vice': 1, 'presid': 1, 'broadcast': 1, 'decemb': 1, '1985': 1, 'under': 1, 'earli': 1, 'retir': 1, 'neithe

<b>For your testing:</b>

In [7]:
assert(len(doc_term_freqs) == 20000)

In [8]:
assert(doc_term_freqs[0]["blair"] == 5)

In [9]:
assert(doc_term_freqs[100]["bank"] == 3)

# 2. Inverted Index (1 mark)

To create an `InvertedIndex` class using `vocab` and `doc_term_freqs` 

Our `InvertedIndex` class contains <b>six</b> components:

1. The vocabulary `vocab`, which will be used to map query terms to term ids
2. The length of each document,  `doc_len`
3. `doc_ids` is a list indexed by term IDs. For each term ID, it stores a list of document ids of all documents containing that term
4. `doc_term_freqs` is a list indexed by term IDs. For each term ID, it stores a list of document term frequencies $f_{d,t}$ (how often a document $d$ contains the term $t$) of the corresponding documents stored in `doc_ids`
5. `doc_freqs` is a list indexed by term IDs. For each term ID, it stores the document frequency $f_t$ indicating the number of documents containing one or more occurrences of term $t$;
6. Two integers `total_num_docs` and `max_doc_len` store the total number of documents and the maximum document length

These values will be used in the code below. Note that some of these components are for display purposes to verify that the implementation was correctly processing the text collection, but won't be used in TF-IDF scoring.

In [10]:
class InvertedIndex:
    def __init__(self, vocab, doc_term_freqs):
        self.vocab = vocab
        self.doc_len = [0] * len(doc_term_freqs)
        self.doc_term_freqs = [[] for i in range(len(vocab))]
        self.doc_ids = [[] for i in range(len(vocab))]
        self.doc_freqs = [0] * len(vocab)
        self.total_num_docs = 0
        self.max_doc_len = 0
        for docid, term_freqs in enumerate(doc_term_freqs):
            doc_len = sum(term_freqs.values())
            self.max_doc_len = max(doc_len, self.max_doc_len)
            self.doc_len[docid] = doc_len
            self.total_num_docs += 1
            for term, freq in term_freqs.items():
                term_id = vocab[term]
                self.doc_ids[term_id].append(docid)
                self.doc_term_freqs[term_id].append(freq)
                self.doc_freqs[term_id] += 1

    def num_terms(self):
        return len(self.doc_ids)

    def num_docs(self):
        return self.total_num_docs

    def docids(self, term):
        term_id = self.vocab[term]
        return self.doc_ids[term_id]

    def freqs(self, term):
        term_id = self.vocab[term]
        return self.doc_term_freqs[term_id]

    def f_t(self, term):
        term_id = self.vocab[term]
        return self.doc_freqs[term_id]

    def space_in_bytes(self):
        # this function assumes each integer is stored using 8 bytes
        space_usage = 0
        for doc_list in self.doc_ids:
            space_usage += len(doc_list) * 8
        for freq_list in self.doc_term_freqs:
            space_usage += len(freq_list) * 8
        return space_usage
    

invindex = InvertedIndex(vocab, doc_term_freqs)

# print inverted index stats
print("documents = {}".format(invindex.num_docs()))
print("number of terms = {}".format(invindex.num_terms()))
print("longest document length = {}".format(invindex.max_doc_len))
print("uncompressed space usage MiB = {:.3f}".format(invindex.space_in_bytes() / (1024.0 * 1024.0)))

documents = 20000
number of terms = 103193
longest document length = 10576
uncompressed space usage MiB = 58.187


<b>Use the `InvertedIndex` class</b>:(1 mark)

To compute the TF-IDF similarity scores for the documents given a simple query $Q$.

Here is a simplified formula for computing TF-IDF similarity scores:

\begin{equation*}
Score(Q,d) = \frac{1}{\sqrt{|d|}} \times \sum_{i=1}^q \log(1 + f_{d,t}) * \log( \frac{N}{f_t} ) 
\end{equation*}

where $Q$ corresponds to a query containing $q$ query terms, $\sqrt{|d|}$ corresponds to the length of the document (in words), $f_{d,t}$ corresponds to the frequency of term $t$ in document $d$, $N$ corresponds to the number of documents in the collection, and $f_t$ corresponds to the document frequency of term $t$. All these information are available in the `InvertedIndex` class. Note that the formulation of TF-IDF is a little different to the formula for TF-IDF in the lectures. We have adapted the formulation here to allow for a simpler implementation, e.g., avoiding the need for repeated passes over the dataset. (All manner of variants of TF-IDF exist in practise.)

Will implement the `query_tfidf` function. The `query_tfidf` function should take a query and an inverted index and output the top $k$ highest scoring documents. 


In [11]:
from math import log, sqrt

# given a query and an index returns a list of the k highest scoring documents as tuples containing <docid,score>
def query_tfidf(query, index, k=10):
    
    # scores stores doc ids and their scores
    scores = Counter()
    
    ###
    # Your answer BEGINS HERE
    ###
    for query_term in query:
        for idx, docid in enumerate(index.docids(query_term)):
            sumCalc = 0
            doc_lenght = (sqrt(index.doc_len[docid]))**(-1)
            sumCalc += log(1 + index.freqs(query_term)[idx]) * \
                    log(index.num_docs()/index.f_t(query_term))
            scores[docid] += doc_lenght * sumCalc 
            
    ###
    # Your answer ENDS HERE
    ###
    
    return scores.most_common(k)


# We output some statistics from our index
query = "south korea production"
stemmed_query = nltk.stem.PorterStemmer().stem(query).split()
results = query_tfidf(stemmed_query, invindex)
for rank, res in enumerate(results):
    # e.g RANK 1 DOCID 176 SCORE 0.426 CONTENT South Korea rose 1% in February from a year earlier, the
    print("RANK {:2d} DOCID {:8d} SCORE {:.3f} CONTENT {:}".format(rank+1,res[0],res[1],raw_docs[res[0]][:75]))

RANK  1 DOCID     1084 SCORE 1.210 CONTENT Unemployment in South Korea fell to 3.8% of the labor force last year from 
RANK  2 DOCID      905 SCORE 1.127 CONTENT Seasonally adjusted industrial production in South Korea increased nearly 2
RANK  3 DOCID     5612 SCORE 1.056 CONTENT Consumer prices in South Korea rose 2.4% in April from a year-earlier and 0
RANK  4 DOCID     9126 SCORE 0.998 CONTENT Foreign investment in South Korea totaled $278 million in 1987's first quar
RANK  5 DOCID    17960 SCORE 0.936 CONTENT Henley Group Inc. said its M.W. Kellogg Co. unit received a contract to des
RANK  6 DOCID     1760 SCORE 0.926 CONTENT Consumer prices in South Korea rose 1% in February from a year earlier, the
RANK  7 DOCID     4132 SCORE 0.926 CONTENT South Korea's trade deficit with Japan grew to a record $629 million in Apr
RANK  8 DOCID    17826 SCORE 0.923 CONTENT South Korea revised its 1986 current-account surplus to $5 billion from the
RANK  9 DOCID    15803 SCORE 0.911 CONTENT South

<b>For testing:</b>

In [12]:
# Rank 1: DOCID
assert(results[0][0] > 500 and results[0][0] < 3000)

In [13]:
# Rank 1: SCORE
assert(results[0][1] > 0.5 and results[0][1] < 2)

## 3. Vbyte compression and decompression (2 marks)

We will reduce the space usage of the inverted index by compression. We will <i>compress</i> the `doc_ids` and `doc_term_freqs` lists in the inverted index using <b>vbyte</b> compression.

<b>Implement two methods</b>: (1 mark) 

To perform vbyte compression and decompression as described in the lecture slides. 
The method signatures are provided below. 

- The first method `vbyte_encode(num)` should receive a number as an integer and produces a list of output bytes encoding the number. 
- The second method `vbyte_decode(input_bytes, idx)` should receive a list of input bytes and an offset into that list where the decompression should start. It returns the decoded number and the number of bytes consumed to decode the number.



In [14]:
def vbyte_encode(num):

    # out_bytes stores a list of output bytes encoding the number
    out_bytes = []
    
    while(num >= 128):
        out_bytes.append(num%128)
        num = num//128
    out_bytes.append(num + 128)  
 
    
    return out_bytes


def vbyte_decode(input_bytes, idx):
    
    # x stores the decoded number
    x = 0
    # consumed stores the number of bytes consumed to decode the number
    consumed = 0
    s = 0
    while(input_bytes[idx]<128):
        x = x^(input_bytes[idx]<<s)
        s = s + 7
        idx+=1
        consumed+=1
    x = x^((input_bytes[idx]-128)<<s)
    consumed = consumed +1    

    return x, consumed


<b>For testing:</b>

In [15]:
# As a sanity check, we ensure that compression and decompression work correctly:
for num in range(0, 123456):
    vb = vbyte_encode(num)
    dec, decoded_bytes = vbyte_decode(vb, 0)
    assert(num == dec)
    assert(decoded_bytes == len(vb))


<b>Modify the `InvertedIndex` class</b>: (1 mark)

To support compression.

Implement the compression of the `doc_ids` and `doc_term_freqs` lists using the `vbyte_encode` function implemented earlier. Note that the `doc_ids` have to be gap encoded as described in the lecture slides.  Will use function `decompress_list` to allow decompression of the lists. 



In [16]:
def decompress_list(input_bytes, gapped_encoded):
    res = []
    prev = 0
    idx = 0
    while idx < len(input_bytes):
        dec_num, consumed_bytes = vbyte_decode(input_bytes, idx)
        idx += consumed_bytes
        num = dec_num + prev
        res.append(num)
        if gapped_encoded:
            prev = num
    return res

class CompressedInvertedIndex:
    def __init__(self, vocab, doc_term_freqs):
        self.vocab = vocab
        self.doc_len = [0] * len(doc_term_freqs)
        self.doc_term_freqs = [[] for i in range(len(vocab))]
        self.doc_ids = [[] for i in range(len(vocab))]
        self.doc_freqs = [0] * len(vocab)
        self.total_num_docs = 0
        self.max_doc_len = 0
        for docid, term_freqs in enumerate(doc_term_freqs):
            doc_len = sum(term_freqs.values())
            self.max_doc_len = max(doc_len, self.max_doc_len)
            self.doc_len[docid] = doc_len
            self.total_num_docs += 1
            for term, freq in term_freqs.items():
                term_id = vocab[term]
                self.doc_ids[term_id].append(docid)
                self.doc_term_freqs[term_id].append(freq)
                self.doc_freqs[term_id] += 1

        # TODO NOW WE COMPRESS THE LISTS

        output_dtf = []
        for idx, term_list in  enumerate(self.doc_term_freqs):
            term_compressed_list = list()
            for num in term_list:
                term_compressed_list.extend(vbyte_encode(num))
            output_dtf.append(term_compressed_list)
        self.doc_term_freqs = output_dtf
        
        
        output_docid = []
        for val, d_id_list in enumerate(self.doc_ids):
            term_compressed_list = list()
            for i in range(0,len(d_id_list)):
                if i == 0:
                    term_compressed_list.extend(vbyte_encode(d_id_list[i]))
                else:
                    diff = (d_id_list[i]) - (d_id_list[i-1])  
                    term_compressed_list.extend(vbyte_encode(diff))
            output_docid.append(term_compressed_list)
        self.doc_ids = output_docid
    
    def num_terms(self):
        return len(self.doc_ids)

    def num_docs(self):
        return self.total_num_docs

    def docids(self, term):
        term_id = self.vocab[term]
        # We decompress
        return decompress_list(self.doc_ids[term_id], True)

    def freqs(self, term):
        term_id = self.vocab[term]
        # We decompress
        return decompress_list(self.doc_term_freqs[term_id], False)

    def f_t(self, term):
        term_id = self.vocab[term]
        return self.doc_freqs[term_id]

    def space_in_bytes(self):
        # this function assumes the integers are now bytes
        space_usage = 0
        for doc_list in self.doc_ids:
            space_usage += len(doc_list)
        for freq_list in self.doc_term_freqs:
            space_usage += len(freq_list)
        return space_usage


# We output the same statistics as before to ensure we still store the same data but now use much less space
compressed_index = CompressedInvertedIndex(vocab, doc_term_freqs)

print("documents = {}".format(compressed_index.num_docs()))
print("unique terms = {}".format(compressed_index.num_terms()))
print("longest document = {}".format(compressed_index.max_doc_len))
print("compressed space usage MiB = {:.3f}".format(compressed_index.space_in_bytes() / (1024.0 * 1024.0)))

documents = 20000
unique terms = 103193
longest document = 10576
compressed space usage MiB = 7.818


<b>For testing:</b>

In [17]:
# Additionally we want to ensure that the index still returns the same results as before
query = "south korea production"
stemmed_query = nltk.stem.PorterStemmer().stem(query).split()
comp_results = query_tfidf(stemmed_query, compressed_index)
for rank, res in enumerate(comp_results):
    print("RANK {:2d} DOCID {:8d} SCORE {:.3f} CONTENT {:}".format(rank+1,res[0],res[1],raw_docs[res[0]][:75]))

RANK  1 DOCID     1084 SCORE 1.210 CONTENT Unemployment in South Korea fell to 3.8% of the labor force last year from 
RANK  2 DOCID      905 SCORE 1.127 CONTENT Seasonally adjusted industrial production in South Korea increased nearly 2
RANK  3 DOCID     5612 SCORE 1.056 CONTENT Consumer prices in South Korea rose 2.4% in April from a year-earlier and 0
RANK  4 DOCID     9126 SCORE 0.998 CONTENT Foreign investment in South Korea totaled $278 million in 1987's first quar
RANK  5 DOCID    17960 SCORE 0.936 CONTENT Henley Group Inc. said its M.W. Kellogg Co. unit received a contract to des
RANK  6 DOCID     1760 SCORE 0.926 CONTENT Consumer prices in South Korea rose 1% in February from a year earlier, the
RANK  7 DOCID     4132 SCORE 0.926 CONTENT South Korea's trade deficit with Japan grew to a record $629 million in Apr
RANK  8 DOCID    17826 SCORE 0.923 CONTENT South Korea revised its 1986 current-account surplus to $5 billion from the
RANK  9 DOCID    15803 SCORE 0.911 CONTENT South